# olmo3 sft dataset

In [52]:

from datasets import load_dataset

dataset_name = "allenai/tulu-3-sft-olmo-2-mixture"
dataset = load_dataset(dataset_name, split="train")




README.md:   0%|          | 0.00/6.20k [00:00<?, ?B/s]

train-00000-of-00006.parquet:   0%|          | 0.00/362M [00:00<?, ?B/s]

train-00001-of-00006.parquet:   0%|          | 0.00/476M [00:00<?, ?B/s]

train-00002-of-00006.parquet:   0%|          | 0.00/147M [00:00<?, ?B/s]

train-00003-of-00006.parquet:   0%|          | 0.00/162M [00:00<?, ?B/s]

train-00004-of-00006.parquet:   0%|          | 0.00/149M [00:00<?, ?B/s]

train-00005-of-00006.parquet:   0%|          | 0.00/116M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/939344 [00:00<?, ? examples/s]

In [53]:
dataset

Dataset({
    features: ['id', 'messages', 'source', 'dataset'],
    num_rows: 939344
})

In [58]:
dataset[0]

{'id': 'oasst1_5921',
 'messages': [{'content': 'Create a snippet of Terraform HCL code that create an AWS autoscaling group, and an ALB in front to expose an application to internet.',
   'role': 'user'},
  {'content': 'Sure, here\'s an example Terraform HCL code that creates an AWS Autoscaling Group and an Application Load Balancer to expose an application to the internet:\n``` \n# Configure the AWS provider\nprovider "aws" {\n  region = "us-east-1"\n}\n\n# Create a security group to allow traffic to the ALB\nresource "aws_security_group" "alb_sg" {\n  name_prefix = "alb_sg"\n  ingress {\n    from_port = 80\n    to_port = 80\n    protocol = "tcp"\n    cidr_blocks = ["0.0.0.0/0"]\n  }\n}\n\n# Create an ALB and target group\nresource "aws_lb" "alb" {\n  name               = "example-alb"\n  internal           = false\n  load_balancer_type = "application"\n\n  subnets = ["subnet-12345678", "subnet-87654321"]\n\n  security_groups = [aws_security_group.alb_sg.id]\n\n  tags = {\n    Enviro

In [60]:
for i in range(5):
    print("USER::::::::::" + dataset[i]['messages'][0]['content'])
    print("ASSISTANT::::::::::" + dataset[i]['messages'][1]['content'])
    print('-'*100)

USER::::::::::Create a snippet of Terraform HCL code that create an AWS autoscaling group, and an ALB in front to expose an application to internet.
ASSISTANT::::::::::Sure, here's an example Terraform HCL code that creates an AWS Autoscaling Group and an Application Load Balancer to expose an application to the internet:
``` 
# Configure the AWS provider
provider "aws" {
  region = "us-east-1"
}

# Create a security group to allow traffic to the ALB
resource "aws_security_group" "alb_sg" {
  name_prefix = "alb_sg"
  ingress {
    from_port = 80
    to_port = 80
    protocol = "tcp"
    cidr_blocks = ["0.0.0.0/0"]
  }
}

# Create an ALB and target group
resource "aws_lb" "alb" {
  name               = "example-alb"
  internal           = false
  load_balancer_type = "application"

  subnets = ["subnet-12345678", "subnet-87654321"]

  security_groups = [aws_security_group.alb_sg.id]

  tags = {
    Environment = "production"
  }
}

resource "aws_lb_target_group" "target_group" {
  name 

# olmo3 DPO dataset

In [ ]:

dataset_name = "allenai/olmo-2-0325-32b-preference-mix"
dataset = load_dataset(dataset_name, split="train")

dataset





README.md:   0%|          | 0.00/5.31k [00:00<?, ?B/s]

train-00000-of-00005.parquet:   0%|          | 0.00/360M [00:00<?, ?B/s]

train-00001-of-00005.parquet:   0%|          | 0.00/260M [00:00<?, ?B/s]

train-00002-of-00005.parquet:   0%|          | 0.00/214M [00:00<?, ?B/s]

train-00003-of-00005.parquet:   0%|          | 0.00/212M [00:00<?, ?B/s]

train-00004-of-00005.parquet:   0%|          | 0.00/214M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/377809 [00:00<?, ? examples/s]

Dataset({
    features: ['chosen', 'rejected', 'chosen_model', 'rejected_model', 'id', 'source'],
    num_rows: 377809
})

In [14]:
dataset

Dataset({
    features: ['chosen', 'rejected', 'chosen_model', 'rejected_model', 'id', 'source'],
    num_rows: 377809
})

In [20]:
dataset[15]

{'chosen': [{'content': 'Translate to English\n\n"Миры Высших Трансмифов Верховных Релгий"',
   'role': 'user'},
  {'content': '"Worlds of Higher Transmissions of Supreme Religions"\n\nThis phrase appears to be describing various spiritual realms or dimensions associated with supreme or highest religious beliefs, and the term "Transmissions" likely refers to the means by which these realms or beliefs are communicated or accessed.',
   'role': 'assistant'}],
 'rejected': [{'content': 'Translate to English\n\n"Миры Высших Трансмифов Верховных Релгий"',
   'role': 'user'},
  {'content': 'The phrase "Миры Высших Трансмифов Верховных Релгий" appears to be a fictional or esoteric term, possibly from a specific book, game, or fictional world. Translating it directly into English would not make sense without context, as it does not correspond to known language or concepts. The translation might be attempted as "Worlds of the Supreme Transmifs of the Ultimate Relgils" or something along similar

## send to polars and search for code

In [47]:
# use polars dataframes, not pandas

import polars as pl

df = dataset.to_polars()

# Extract the first message content from the chosen column
df = df.with_columns(
    pl.col("chosen").list.get(0).struct.field("content").alias("first_message")
)

# Keywords that might indicate Python code
python_indicators = [
    r"```python\b", # Code blocks explicitly marked as Python
    r"```py\b",     # Alternative Python code block marker
    r"\bdef\s+[a-zA-Z_][a-zA-Z0-9_]*\s*\(", # Function definitions
    r"\bclass\s+[A-Z][a-zA-Z0-9_]*\s*[:\(]", # Class definitions
    r"\bimport\s+[a-zA-Z_][a-zA-Z0-9_]*", # Import statements
    r"\bfrom\s+[a-zA-Z_][a-zA-Z0-9_.]*\s+import\b", # From imports
    r"\bif\s+__name__\s*==\s*['\"]__main__['\"]", # Main guard
    r">>>\s*[a-zA-Z_]", # Python REPL
    r"\bassert\s+[^.]+", # Assert statements
    r"\byield\s+[a-zA-Z_]", # Yield statements
    r"@[a-zA-Z_][a-zA-Z0-9_]*\s*[\n\(]", # Decorators
    r"\bawait\s+[a-zA-Z_]", # Await expressions
    r"\basync\s+def\b", # Async function definitions
]


# Filter and display Python-related examples
python_filter = pl.any_horizontal(
    *[pl.col("first_message").str.contains(pattern, literal=False) for pattern in python_indicators]
)
python_examples = df.filter(python_filter)

print(f"Found {len(python_examples)} potential Python code examples")


Found 11306 potential Python code examples


In [49]:
print("\nFirst few examples:")
print(python_examples.select(["first_message"]).head(3))


First few examples:
shape: (3, 1)
┌─────────────────────────────────┐
│ first_message                   │
│ ---                             │
│ str                             │
╞═════════════════════════════════╡
│ I have functions:               │
│ portfolio_me…                   │
│   const doLogOut = async () =>… │
│ how can I call this audio <aud… │
└─────────────────────────────────┘


In [51]:
# Print the first five first_messages in python_examples
print("\nFirst five Python examples:")
for i, msg in enumerate(python_examples.select("first_message").head(5).to_series()):
    print(f"\nExample {i+1}:\n{msg}")


First five Python examples:

Example 1:
I have functions:
portfolio_mean(weights; r_mean = r_mean) =  sum(weights .* r_mean')
portfolio_var(weights; covar = covar) = weights'*covar*weights
and I want to optimize them like that:
function opt_portfolio_both(rates_of_return, rmin,var_max)
    portfolio = Model(Ipopt.Optimizer)
    set_silent(portfolio)
    @variable(portfolio, x[1:size(rates_of_return)[2]] >= 0)
    @objective(portfolio, Max, (portfolio_mean(x)/portfolio_var(x)))
    @constraint(portfolio, sum(x) == 1)
    @constraint(portfolio, portfolio_mean(x) >= r_min)
    @constraint(portfolio, portfolio_var(x) <= var_max);
    optimize!(portfolio)
    objective_value(portfolio)
    weights_opt = value.(x);
    return weights_opt
end
in Julia, but when I try to run it I get error:
/(::Int64,::QuadExpr) is not defined. Are you trying to build a nonlinear problem? Make sure you use @NLconstraint/@NLobjective. If you are using an `@NL` macro and you encountered this error message, it i